## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [ ]:
from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

In [ ]:
# A seguir um exemplo de query na tabela Albums
# usamos o objeto Base para acessar cada modelo ORM.

session = Session(engine)
res = session.scalars(select(Base.classes.albums))
first_album = res.first()
print(first_album.AlbumId, first_album.Title)

#### Q3. 
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [ ]:
from sqlalchemy import select

session = Session(engine)
tracks = session.execute(select(Base.classes.tracks).limit(3)).fetchall()
for track in tracks:
    print(track)


In [ ]:
from sqlalchemy import create_engine, MetaData, select
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

# Conectar ao banco de dados SQLite
engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

# Refletir a estrutura do banco de dados para carregar as tabelas
metadata = MetaData()
metadata.reflect(engine)

# Mapear as tabelas como classes
Base = automap_base(metadata=metadata)
Base.prepare()

# Criar uma sessão para consultar o banco de dados
session = Session(engine)

# Imprimir o nome da faixa e o título do álbum das primeiras 20 faixas na tabela 'tracks'
tracks_albums = session.execute(
    select(Base.classes.tracks.Name, Base.classes.albums.Title)
    .join(Base.classes.albums, Base.classes.tracks.AlbumId == Base.classes.albums.AlbumId)
    .limit(20)
).fetchall()

# Exibir os resultados
print("Nome da faixa e título do álbum das primeiras 20 faixas:")
for track_album in tracks_albums:
    print(f"Faixa: {track_album[0]}, Álbum: {track_album[1]}")

# Fechar a sessão
session.close()


In [ ]:
from sqlalchemy import create_engine, MetaData, select
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

# Conectar ao banco de dados SQLite
engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

# Refletir a estrutura do banco de dados para carregar as tabelas
metadata = MetaData()
metadata.reflect(engine)

# Mapear as tabelas como classes
Base = automap_base(metadata=metadata)
Base.prepare()

# Criar uma sessão para consultar o banco de dados
session = Session(engine)

# Imprimir os nomes das faixas vendidas e a quantidade vendida das 10 primeiras vendas
invoice_items_tracks = session.execute(
    select(Base.classes.tracks.Name, Base.classes.invoice_items.Quantity)
    .join(Base.classes.tracks, Base.classes.invoice_items.TrackId == Base.classes.tracks.TrackId)
    .limit(10)
).fetchall()

# Exibir os resultados
print("Nomes das faixas vendidas e a quantidade vendida das 10 primeiras vendas:")
for track_item in invoice_items_tracks:
    print(f"Faixa: {track_item[0]}, Quantidade Vendida: {track_item[1]}")

# Fechar a sessão
session.close()



In [ ]:
from sqlalchemy import create_engine, MetaData, select, func
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

# Conectar ao banco de dados SQLite
engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

# Refletir a estrutura do banco de dados para carregar as tabelas
metadata = MetaData()
metadata.reflect(engine)

# Mapear as tabelas como classes
Base = automap_base(metadata=metadata)
Base.prepare()

# Criar uma sessão para consultar o banco de dados
session = Session(engine)

# Imprimir os nomes das 10 faixas mais vendidas e a quantidade de vezes que foram vendidas
top_tracks = session.execute(
    select(Base.classes.tracks.Name, func.sum(Base.classes.invoice_items.Quantity).label('Total_Vendido'))
    .join(Base.classes.invoice_items, Base.classes.tracks.TrackId == Base.classes.invoice_items.TrackId)
    .group_by(Base.classes.tracks.TrackId)
    .order_by(func.sum(Base.classes.invoice_items.Quantity).desc())
    .limit(10)
).fetchall()

# Exibir os resultados
print("Top 10 faixas mais vendidas e a quantidade vendida:")
for track in top_tracks:
    print(f"Faixa: {track[0]}, Quantidade Vendida: {track[1]}")

# Fechar a sessão
session.close()


In [ ]:
from sqlalchemy import create_engine, MetaData, select, func
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

# Conectar ao banco de dados SQLite
engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

# Refletir a estrutura do banco de dados para carregar as tabelas
metadata = MetaData()
metadata.reflect(engine)

# Mapear as tabelas como classes
Base = automap_base(metadata=metadata)
Base.prepare()

# Criar uma sessão para consultar o banco de dados
session = Session(engine)

# Imprimir os 10 artistas que mais venderam
top_artists = session.execute(
    select(Base.classes.artists.Name, func.sum(Base.classes.invoice_items.Quantity).label('Total_Vendido'))
    .join(Base.classes.tracks, Base.classes.invoice_items.TrackId == Base.classes.tracks.TrackId)
    .join(Base.classes.albums, Base.classes.tracks.AlbumId == Base.classes.albums.AlbumId)
    .join(Base.classes.artists, Base.classes.albums.ArtistId == Base.classes.artists.ArtistId)
    .group_by(Base.classes.artists.ArtistId)
    .order_by(func.sum(Base.classes.invoice_items.Quantity).desc())
    .limit(10)
).fetchall()

# Exibir os resultados
print("Top 10 artistas que mais venderam:")
for artist in top_artists:
    print(f"Artista: {artist[0]}, Quantidade Vendida: {artist[1]}")

# Fechar a sessão
session.close()
